写一个cache装饰器，允许过期，但没有换出，没有清除

1.缓存需要有几个元素

相同的函数，相同参数，相同的参数值

In [29]:
d = {}

In [30]:
cache = []

In [63]:
import functools
import inspect
import datetime

def cache(exp=0):
    def _cache(fn):
        cache = {}
        @functools.wraps(fn)
        def wrap(*args, **kwargs):
            # TODO key如何拼装
            key = []

            names = set()

            params = inspect.signature(fn).parameters

            # 位置参数
            for i, arg in enumerate(args):
                name = list(params.keys())[i]
                key.append((name, arg))
                names.add(name)

            # 关键字参数
            key.extend(kwargs.items())
            names.update(kwargs.keys())

            for k, v in params.items():
                if k not in names:
                    key.append((k, v.default))

            key.sort(key=lambda x: x[0])

            key = '&'.join(['{}={}'.format(name, arg) for name, arg in key])

            print(key)

            if key in cache.keys():
                # TODO 超时时间
                ret, timestamp = cache[key]
                if exp == 0 or datetime.datetime.now().timestamp() - timestamp < exp:
                    print('cache hit')
                    return ret
                
            ret = fn(*args, **kwargs)
            print('cache miss')
            cache[key] = (ret, datetime.datetime.now().timestamp())
            return ret
        return wrap
    return _cache

In [65]:
@cache(5)
def add(x ,y=3):
    return x + y

In [80]:
add(1, 3)

x=1&y=3
cache miss


4

In [22]:
add(1, 3)

4

In [23]:
add(2, 2)

4

In [26]:
def plus(x, y):
    return x * y

In [28]:
plus(2, 2)

4

In [2]:
import inspect

In [44]:
def add(x: int, y: int):
    return x + y

In [45]:
sig = inspect.signature(add)

In [49]:
sig.parameters.keys()

odict_keys(['x', 'y'])

In [48]:
sig.parameters['x'].name

'x'

In [12]:
import functools
import time

In [13]:
@functools.lru_cache(3)
def sleep(x):
    time.sleep(x)

In [14]:
sleep(3)

In [15]:
sleep(4)

In [16]:
sleep(2)

In [17]:
sleep(5)

In [18]:
sleep(4)

In [19]:
sleep(3)

写一个通用的命令分发器，随时能够注册命令到我们的分发器里

In [88]:
def add(x , y):
    return x + y

In [83]:
input()

add


'add'

In [84]:
input()

quit


'quit'

In [85]:
input()

add(1, 3)


'add(1, 3)'

In [86]:
commands = {}

In [121]:
def commands():
    cmds = {}
    def register(command):
        def _register(fn):
            cmds[command] = fn
            return fn
        return _register

    def default_fn():
        print('undefined command')

    def run():
        while True:
            cmd = input('>>')
            if cmd.strip() == 'quit':
                return
            cmds.get(cmd.strip(), default_fn)()
    
    return register, run

In [122]:
a,  = commands()

In [125]:
@a('papapa')
def papapa():
    print('papapa')

In [126]:
b()

>>papapa
papapa
>>quit


In [114]:
d = {'a': 1}

In [115]:
d.get('b', 2)

2

In [89]:
commands['add'] = add

In [92]:
commands['add'](1, 3)

4

In [93]:
def default_fn():
    print('undefined command')

In [94]:
def run():
    while True:
        cmd = input('>>')
        if cmd.strip() == 'quit':
            return
        commands.get(cmd.strip(), default_fn())

In [95]:
register('add', add)

In [96]:
commands

{'add': <function __main__.add>}

In [97]:
run()

>>ssdf
undefined command
>>add
undefined command
>>add(1, 3)
undefined command
>>quit
